### Библиотеки / данные 

импортируем numpy и pandas

In [ ]:
import numpy as np
import pandas as pd

настройки pandas

In [ ]:
pd.options.display.max_rows = 10

- считываем данные
- используем столбец Symbol в качестве индекса 
- считываем только столбцы ['Symbol', 'Sector', 'Price', 'Book Value']

| Column Name        | Description
| ------------- |:-------------:|
|Symbol|Сокращенное название организации|
|Name|Полное название организации|
|Sector|Сектор экономики|
|Price|Стоимость акции|
|Dividend Yield|Дивидендная доходность|
|Price/Earnings|Цена / прибыль|
|Earnings/Share|Прибыль на акцию|
|Book Value|Балансовая стоимость компании|
|52 week low|52-недельный минимум|
|52 week high|52-недельный максимум|
|Market Cap|Рыночная капитализация|
|EBITDA|**E**arnings **b**efore **i**nterest, **t**axes, **d**epreciation and **a**mortization|
|Price/Sales|Цена / объём продаж|
|Price/Book|Цена / балансовая стоимость|
|SEC Filings|Ссылка *sec.gov*|

In [ ]:
sp500 = pd.read_csv("sp500.csv",
                    index_col='Symbol', 
                    usecols=['Symbol', 'Sector', 'Price', 'Book Value'])
sp500

### Операции

#### сброс индекса

сбрасываем индекс, помещая значения индекса в столбец

In [ ]:
index_moved_to_col = sp500.reset_index()
index_moved_to_col.head()

#### установка

а теперь делаем столбец Sector индексом

In [ ]:
index_moved_to_col.set_index('Sector').head()

#### операции над множествами

Датафреймы для примера:

In [ ]:
data_rnd_part_1 = sp500.sample(100, random_state=333)
data_rnd_part_2 = sp500.sample(100, random_state=444)

In [ ]:
data_rnd_part_1.head()

In [ ]:
data_rnd_part_2.head()

объединение:

In [ ]:
ind_union = data_rnd_part_1.index.union(data_rnd_part_2.index)
ind_union

пересечение

In [ ]:
ind_intersection = data_rnd_part_1.index.intersection(data_rnd_part_2.index)
print(ind_intersection)
print(len(ind_intersection))

разность

In [ ]:
ind_diff = data_rnd_part_1.index.difference(data_rnd_part_2.index)
len(ind_diff)

### Иерархическая индексация

сначала убираем индекс

In [ ]:
reindexed = sp500.reset_index()

In [ ]:
reindexed.head()

а теперь установим индексацию датафрейма сразу по двум столбцам: Sector и Symbol

In [ ]:
multi_fi = reindexed.set_index(['Sector', 'Symbol'])
multi_fi.head()

наш индекс - это MultiIndex

In [ ]:
type(multi_fi.index)

он имеет два уровня

In [ ]:
len(multi_fi.index.levels)

каждый уровень индекса - это индекс

In [ ]:
multi_fi.index.levels[1]

изменение порядка уровней индекса:

In [ ]:
multi_fi.reorder_levels([1, 0], axis=0).head()

получаем все акции, которые имеют значение Industrials <br> обратите внимание, что в результатах индекс уровня 0 не выводится 

In [ ]:
multi_fi.xs('Industrials').head()

отбираем строки, в которых индекс уровня 1 (Symbol) имеет значение ALLE

In [ ]:
multi_fi.xs('ALLE', level=1)

скомбинируем уровни индексов

In [ ]:
multi_fi.xs('Industrials').xs('UPS')

комбинируем уровни индексов, используя кортеж

In [ ]:
multi_fi.xs(('Industrials', 'UPS'))

создаем датафрейм с 5 строками и 3 столбцами

In [ ]:
df = pd.DataFrame(np.arange(0, 15).reshape(5, 3), 
                  index=['a', 'b', 'c', 'd', 'e'], 
                  columns=['c1', 'c2', 'c3'])
df

- добавляем несколько столбцов и строк в датафрейм столбец c4 со значениями NaN
- строка 'f' со значениями от 15 до 18 
- строка 'g', состоящая из значений NaN
- столбец 'c5', состоящий из значений NaN
- меняем значение в столбце 'c4' строки 'a'

In [ ]:
df['c4'] = np.nan
df.loc['f'] = np.arange(15, 19) 
df.loc['g'] = np.nan
df['c5'] = np.nan
df['c4']['a'] = 20
df

### Работа с пропущенными значениями

#### поиск

какие элементы являются значениями NaN?

In [ ]:
df.isnull()

какие элементы являются непропущенными значениями? (можем использовать ~df.isnull() )

In [ ]:
df.notnull()

подсчитываем количество значений NaN в каждом столбце

In [ ]:
df.isnull().sum(axis=0)

вычисляем количество значений, отличных от NaN, по каждому столбцу (можем использовать len(df) - df.isnull().sum())


In [ ]:
df.count(axis=0)

#### удаление

In [ ]:
df

отбираем непропущенные значения в столбце c4

In [ ]:
df.c4[df.c4.notnull()]

этот программный код извлекает в столбце c4 все значения, кроме значений NaN

In [ ]:
df.c4.dropna()

.dropna() возвращает копию с удаленными значениями исходный датафрейм/столбец не изменился

In [ ]:
df.c4

метод .dropna() при применении к датафрейму удаляет целиком строки, в которых есть по крайней мере одно значение NaN в данном случае будут удалены все строки

In [ ]:
df.dropna()

используя параметр how='all', удаляем лишь те строки, в которых все значения являются значениями NaN

In [ ]:
df.dropna(how = 'all')

In [ ]:
df

меняем ось, чтобы удалить столбцы со значениями NaN вместо строк

In [ ]:
df.dropna(how='all', axis=1) # удаляем c5

- создаем копию датафрейма df
- заменяем две ячейки с пропусками значениями 0

In [ ]:
df2 = df.copy()
df2.loc['g'].c1 = 0
df2.loc['g'].c3 = 0
df2

а сейчас удаляем столбцы, в которых есть хотя бы одно значение NaN

In [ ]:
df2.dropna(how='any', axis=1) 

#### заполнение

##### константой

In [ ]:
df

возвращаем новый датафрейм, в котором значения NaN заполнены константой - нулями

In [ ]:
filled = df.fillna(0)
filled

значения NaN не учитываются при вычислении средних значений

In [ ]:
df.mean()

после замены значений NaN на 0 получаем другие средние значения

In [ ]:
filled.mean()

##### прямое и обратное

заполнение в прямом порядке

In [ ]:
df.c4

In [ ]:
df.c4.fillna(method="ffill")

либо выполняем обратное заполнение

In [ ]:
df.c4.fillna(method="bfill")

##### с помощью индексов

заполняем значения NaN в каждом столбце средним значением этого столбца

In [ ]:
df.mean()

In [ ]:
df

In [ ]:
df.fillna(df.mean())

#### интерполяция пропущенных значений

выполняем линейную интерполяцию ( method = 'linear' по умолчанию) значений NaN с 1 по 2

In [ ]:
s = pd.Series([1, np.nan, np.nan, np.nan, 2])
s

In [ ]:
s.interpolate()

создаем объект Series, чтобы продемонстрировать интерполяцию, основанную на индексных метках

In [ ]:
s = pd.Series([0, np.nan, 100], index=[0, 2, 10])
s

выполняем линейную интерполяцию

In [ ]:
s.interpolate()

выполняем интерполяцию на основе значений индекса

In [ ]:
s.interpolate(method="index")

### Повторяющиеся значения 

создаем датафрейм с дублирующимися строками

In [ ]:
data = pd.DataFrame({'a': ['x'] * 3 + ['y'] * 4, 
                     'b': [1, 1, 2, 3, 3, 4, 4]})
data

определяем, какие строки являются дублирующимися, то есть какие строки уже ранее встречались в датафрейме

In [ ]:
data.duplicated()

удаляем дублирующиеся строки, каждый раз оставляя первое из дублирующихся наблюдений

In [ ]:
data.drop_duplicates()

удаляем дублирующиеся строки, каждый раз оставляя последнее из дублирующихся наблюдений

In [ ]:
data.drop_duplicates(keep='last')

добавляем столбец:

In [ ]:
data

In [ ]:
data['c'] = range(7)
data.duplicated()

In [ ]:
data

но если мы укажем, что нужно удалить дублирующиеся строки с учетом значений в столбцах a и b, результаты будут выглядеть так

In [ ]:
data.drop_duplicates(['a', 'b'])

### Статистики

In [ ]:
sp500.head()

считываем исторические данные о котировках акций

In [ ]:
omh = pd.read_csv('omh.csv')
print(omh)

omh.set_index('Date', 
              inplace=True)

In [ ]:
omh.head()

### Сводка статистик

получаем сводку статистик для датафрейма, с которой работаем как с обычным датафреймом

In [ ]:
sp500.describe()

вычисляем сводку статистик для отдельного столбца Price

In [ ]:
sp500.Price.describe()

получаем сводку статистик для нечисловых данных

In [ ]:
sp500.Sector.describe()

метод info:

In [ ]:
sp500.info()

получаем сводную статистику для нечисловых данных

In [ ]:
sp500.Sector.value_counts(normalize=True)

### Арифметические операции

- задаем стартовое значение генератора случайных чисел для получения воспроизводимых результатов
- создаем объект DataFrame

In [ ]:
np.random.seed(123)
df = pd.DataFrame(np.random.randn(5, 4), 
                  columns=['A', 'B', 'C', 'D'])
df

умножаем все на 2, берём только абсолютные значения

In [ ]:
abs(df * 2)

вычитаем первую строку из каждой строки объекта DataFrame

In [ ]:
df

In [ ]:
df.iloc[0]

In [ ]:
df - df.iloc[0]

вычитаем объект DataFrame из объекта Series

In [ ]:
df.iloc[0] - df

### Одномерные статистики

#### минимум / максимум

определяем максимальную цену для обеих акций

In [ ]:
omh[['MSFT', 'AAPL']].max()

определяем индекс, которому соответствует максимальная цена для обеих акций

In [ ]:
omh[['MSFT', 'AAPL']].idxmax()

#### cреднее значение / медиана / мода

<img src='..\images\moda-mediana.jpg'/>

вычисляем среднее значение для всех столбцов в датафрейме omh

In [ ]:
omh.mean()

вычисляем значение, усредненное по всем столбцам, для каждой строки (выведем первые 5)

In [ ]:
omh.mean(axis=1).head() 

вычисляем медиану значений для каждого столбца

In [ ]:
omh.median()

вычисляем моду для столбца Sector

In [ ]:
sp500.Sector.mode()

мод может быть несколько, поэтому результат операции - Series 

In [ ]:
s = pd.Series([1, 2, 3, 3, 5, 1])
s.mode()

#### [дисперсия](https://ru.wikipedia.org/wiki/Дисперсия_случайной_величины) / среднеквадратичное отклонение

вычисляем дисперсию значений в каждом столбце

In [ ]:
omh.var()

In [ ]:
(omh.MSFT**2 - omh.MSFT.mean()**2).sum() / (omh.shape[0]-1)

вычисляем среднеквадратичное отклонение

In [ ]:
omh.std()

In [ ]:
omh.MSFT.var()**0.5

#### [ковариация](https://ru.wikipedia.org/wiki/Ковариация) / [корреляция](https://ru.wikipedia.org/wiki/Корреляция)

вычисляем ковариацию между MSFT и AAPL

In [ ]:
omh.MSFT.cov(omh.AAPL)

вычисляем корреляцию между MSFT и AAPL

In [ ]:
omh.MSFT.corr(omh.AAPL)

In [ ]:
omh.MSFT.cov(omh.AAPL) / (omh.MSFT.std() * omh.AAPL.std())

либо можем получать матрицу ковариаций

In [ ]:
omh.corr()